In [1]:
import git2vectors
import chat_utils

c:\Users\voyno\Desktop\code\repos\repo-chat\venv\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
repo = "https://github.com/hwchase17/langchain"
vectorstore = git2vectors.create_vectorstore(repo)

Fetching data from git repo...
Clearing index testindex from previous runs...
Done!

describe_index_stats:
{'dimension': 1536,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 4196}},
 'total_vector_count': 4196}


In [7]:
chain = chat_utils.construct_chain(git2vectors.OPENAI_API_KEY)

Q = "How do I use the Langchain library? Give an example."
inputs, docs = chat_utils.get_chain_inputs(vectorstore, Q)

chat_utils.chat(chain, inputs)